In [1]:
# Import Built in modules
import json # JSON files utilities
from urllib.request import urlretrieve # retrieve files from urls
import os # operative system utilities
import re # regular expressions

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import pandas as pd
import geopandas as gpd
import numpy as np

import ipywidgets as ipw
from ipywidgets import interact,interactive,Layout

In [2]:
# Import my backend module
from AppBackend import processdata

In [3]:
# Function to make a plot of the evolution of COVID-19 in a autonomous community
def plot_CCAA_evolution_plot(CCAA_name,communities_data_frames_dict,ax=None):
    
    try:
        df = communities_data_frames_dict[CCAA_name]
    except:
        print('¡Introduzca una comunidad autónoma válida!')
        return None
    
    # Create a figure if no axes are provided
    if not ax:
        fig, ax = plt.subplots(constrained_layout=True);
    
    # Make the plot  
    df.plot.line(
        x = date_column_name,
        y = [
            cases_column_name
            ,deaths_column_name
        ],
        color = [
            'blue',
            'red'
        ],
        ax = ax
    )
    
    # Set grid for the plot
    # ax.grid(True)
    
    # ticks rotation
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    # set ticks every week
    ax.xaxis.set_major_locator(mdates.WeekdayLocator())
    # set major ticks format
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter(''))
    
    # axes labels
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    # plot title
    ax.set_title(CCAA_name)
    
    return None

In [4]:
# Function to make a stacked area plot of the evolution of COVID-19 in a autonomous community
def plot_CCAA_area_plot(CCAA_name,communities_data_frames_dict,ax=None):
    
    try:
        df = communities_data_frames_dict[CCAA_name]
    except:
        print('¡Introduzca una comunidad autónoma válida!')
        return None
    
    # Create a figure if no axes are provided
    if not ax:
        fig, ax = plt.subplots(constrained_layout=True);
    
    # Make the plot  
    df.plot.area(
        x = date_column_name,
        y = [
            activeCases_column_name
            ,deaths_column_name
            ,recovered_column_name
        ],
        color = [
            'blue',
            'red',
            'green'
        ],
        ax = ax
    )
    
    # Set grid for the plot
    # ax.grid(True)
    
    # ticks rotation
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    # set ticks every week
    ax.xaxis.set_major_locator(mdates.WeekdayLocator())
    # set major ticks format
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter(''))
    
    # axes labels
    ax.set_xlabel('')
    ax.set_ylabel('Casos')
    
    # plot title
    ax.set_title(CCAA_name)
    
    return None

In [5]:
# Function to make a map chart of the actual situation in Spain

def plotMap(column_name,merged_data,ax = None): 
    
    # Values range for the color maps
    vmin = min(merged_data[column_name])
    vmax = max(merged_data[column_name])
    
    # Color map for the plots
    cmap = 'YlGn'

    # Create a figure if no axes are provided
    if not ax:
        fig, ax = plt.subplots(constrained_layout=True);   
    
    # title for the plot
    ax.set_title(column_name)
    # face color of the plot
    ax.set_facecolor('#00FFFF');
    # deactivate grid
    ax.grid(False)
    # remove the ticks
    plt.xticks([],[])
    plt.yticks([],[])

    # Make the plot
    merged_data.plot(
        column=column_name,
        cmap=cmap,
        edgecolors='lightblue',
        vmin = vmin,
        vmax = vmax,
        ax=ax
    );

    sm = plt.cm.ScalarMappable(cmap=cmap,norm=plt.Normalize(vmin=vmin,vmax=vmax))
    fig.colorbar(sm,ax=ax,orientation='horizontal');


In [6]:
# Tuples to store the names and codes of the CCAA
CCAA_dict = processdata.make_CCAA_dict()
CCAA_names_tuple = tuple( sorted(CCAA_dict.keys()) )
CCAA_codes_tuple = tuple( sorted(CCAA_dict.values()) )

# Dictionary relating "cartographic ID numbers":"ISO CODES"
CCAA_cartodb_ID_dict = processdata.make_CCAA_cartodb_ID_dict()

In [7]:
#                Script to obtain COVID data in SPAIN
#
# - data_COVID19_spain = data frame with all the data for spain
# - data_COVID19_spain_last = data frame with the last update
# - commcommunities_data_frames_dict = dictionary containing data frames for 
#                                    every community. {'name':dataframe.}
# - data_COVID19_spain_sum = data frame for the total data in Spain, i.e., 
#                            summed over the communities.


# url for the data and name for the csv file
data_url = 'https://covid19.isciii.es/resources/serie_historica_acumulados.csv'
data_file_name = 'data.csv'

# Download the data csv file and make the national and regional data frames
processdata.downloadDATA(data_url,data_file_name)
data_COVID19_spain = processdata.makeNationalDataFrame()
data_COVID19_columns = data_COVID19_spain.columns.to_list()

# Column names for the data
ISO_code_column_name = data_COVID19_columns[0]
date_column_name = data_COVID19_columns[1]
cases_column_name = data_COVID19_columns[2]
Hospitalized_column_name = data_COVID19_columns[3]
UCI_column_name = data_COVID19_columns[4]
deaths_column_name = data_COVID19_columns[5]
recovered_column_name = data_COVID19_columns[6]
activeCases_column_name = 'Casos Activos'

# Make a column for the ACTIVE cases
data_COVID19_spain[activeCases_column_name] = data_COVID19_spain[cases_column_name] - data_COVID19_spain[deaths_column_name] - data_COVID19_spain[recovered_column_name]

# Data set with the last update
data_COVID19_spain_last = pd.DataFrame(data_COVID19_spain[data_COVID19_spain[date_column_name]==max(data_COVID19_spain[date_column_name])])
data_COVID19_spain_last.reset_index(drop=True,inplace=True)

# Dictionary with a dataframe for everey comunidad autonoma
communities_data_frames_dict = processdata.makeCommunitiesDataFrameDict(data_COVID19_spain)

# Data set with the sum of cases for every community
dates_list = list(dict.fromkeys(data_COVID19_spain[date_column_name]))
dates_list.sort()
data_COVID19_spain_sum = pd.DataFrame(
    {
        date_column_name:dates_list,
        cases_column_name: sum( df[cases_column_name] for df in communities_data_frames_dict.values() ),
        deaths_column_name: sum( df[deaths_column_name] for df in communities_data_frames_dict.values() ),
        recovered_column_name: sum( df[recovered_column_name] for df in communities_data_frames_dict.values() ),
        activeCases_column_name: sum( df[activeCases_column_name] for df in communities_data_frames_dict.values() )
    }
)

In [8]:
#                Script to make a geoDataFrame for the spain geometry

# Data set with the mapa data
map_df = gpd.read_file('shapefiles_espana_ccaa_1')
# pop columns that I don't need
map_df.pop('codaut_sin');
map_df.pop('codaut');

# pop the column with the cartographic CCAA codes
map_df_cartoID_column = map_df.pop('cartodb_id')
# transform it into CCAA ISO codes
ISO_code_column_data = [CCAA_cartodb_ID_dict[x] for x in map_df_cartoID_column]

map_df.insert(0, ISO_code_column_name, ISO_code_column_data, True)

In [9]:
#                Script to merge the geoDATAFrame with the DATA frame of the lastupdate

# merge the coronavirus data set and the map data set
merged_data = map_df.set_index(ISO_code_column_name).join(data_COVID19_spain_last.set_index(ISO_code_column_name))
merged_data.reset_index(inplace=True);

In [10]:
#                                    Making the App Layout

In [11]:
# html header with the styles and fonts awesome icons

In [12]:
%%html
<head>
    <script src="https://use.fontawesome.com/8a19d087db.js"></script>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
    <style>
        a {
            color : blue;
        }
        a:hover {
            text-decoration: underline;
            color : blue;
            
        }
        
        .social {
            text-align: center;
            color: black;
            overflow: hidden;
            
            a {
                text-decoration: none;
            };
        }
        
        i { 
            color: black;
            opacity:0.7;
            padding-top: 5px;
            padding-bottom: 5px;
            padding-left: 10px;
            padding-right: 10px;
            transition:0.3s;
        }

        i:hover {
            opacity: 1;
        }        
        
        i.fa-twitter:hover {
            color:#77DDF6;
        }
        
        i.fa-facebook-official:hover {
            color: #4267B2;
        }
            
        i.fa-envelope:hover {
            opacity :0.7
        }
            
        i.fa-linkedin:hover {
            color: #2867B2;
        }
        i.fa-github:hover {
            color: #211F1F;
        }
            
        .app-header { 
            background-color:black !important; 
            color:white;
            text-align:center;
        }
        
        .bottom-text {
            text-align: center
        }
        
    </style>
</head>

In [13]:
#                                    Left Panel of the App

# Widget to display the evolution of COVID-19 in a  CCAA

# Make a selection widget for the CCAA
CCAA_selector = ipw.Dropdown(
    options = CCAA_names_tuple,
    value=CCAA_names_tuple[0],
    description='',
    disabled = False,
    continuous_update = False
)
# Label for the widget
CCAA_selector_label = ipw.Label('Comunidad autónoma: ')
# Interactive widget to plot the evolution of COVID-19 in a CCAA
w1 = interactive(
    lambda x: plot_CCAA_area_plot(x,communities_data_frames_dict),
    x=CCAA_selector
)
CCAA_selector.description = ''
# The finished widget for the evolution
evol_widget = ipw.VBox([CCAA_selector_label,w1])


# Widget to display the chart map

map_options_label = ipw.Label('Opción: ')

map_options = [
    cases_column_name,
    activeCases_column_name,
    Hospitalized_column_name,
    UCI_column_name,
    deaths_column_name,
    recovered_column_name
]

# widget to select the data to display
map_selector = ipw.Dropdown(
    options = map_options,
    layout = {
        'width': 'max-content'
    },
    disabled=False
)

# map widget
map_widget = ipw.VBox(
    [
        map_options_label,
        interactive(
            lambda x: plotMap(x,merged_data),
            x = map_selector
        )
    ]
)
map_selector.description=''

left_panel = ipw.VBox(
    [map_widget,evol_widget],
    layout = Layout(
        grid_area = 'left-panel'
    )
)
left_panel.layout.align_items = 'center'
left_panel.layout.align_content = 'center'
# left_panel

In [14]:
#                       Right Panel of the App

info_widget = ipw.HTML(
    value = """
    <div>
        <ul>
            <li>
            Los datos han sido extraidos directamente de la <a href = "https://covid19.isciii.es/">página web</a> del ministerio de sanidad que informa de la situación del COVID-19 en España: <a href = "https://covid19.isciii.es/resources/serie_historica_acumulados.csv">https://covid19.isciii.es/resources/serie_historica_acumulados.csv</a>
            </li>
        </ul>
    </div>
    """
)

share_on_SNN_widget = ipw.HTML(
"""
<div class="social">
    <h1>Comparte en RRSS</h1>
    <a href="https://twitter.com/intent/tweet?text=Widget%20para%20seguir%20la%20evoluci%C3%B3n%20del%20COVID-19%20por%20comunidades%20auton%C3%B3micas%3A%0A%0Ahttps%3A//mybinder.org/v2/gh/Miguel-ASM/COVID-19-SPAIN-voila/master?urlpath=%252Fvoila%252Frender%252Findex.ipynb">
        <i id="twitter" class="fa fa-twitter fa-4x wow" ></i></a>
    <a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//mybinder.org/v2/gh/Miguel-ASM/COVID-19-SPAIN-voila/master?urlpath=%252Fvoila%252Frender%252Findex.ipynb">
        <i class="fa fa-facebook-official fa-4x" ></i></a>
</div>
"""
)

follow_author_widget = ipw.HTML(
"""
<div class="social">
    <h1>Creado por</h1>
    <h3>Miguel Ángel Simón Martínez</h3>
    <a href="https://twitter.com/miguel_simar">
        <i class="fa fa-twitter fa-4x" ></i></a>
    <a href="https://www.linkedin.com/in/miguel-%C3%A1ngel-sim%C3%B3n-mart%C3%ADnez-0a457b18b/">
        <i class="fa fa-linkedin fa-4x" ></i></a>
    <div>
        <i class="fa fa-envelope" ></i>
        <font color='blue'>miguel.a.s.martinez@gmail.com</font>
    </div>
</div>

"""
)

source_code_widget = ipw.HTML(
"""
<div class="social">
    <h1>Código fuente</h1>
    <a href="https://github.com/Miguel-ASM/COVID-19-SPAIN-voila">
        <i class="fa fa-github fa-4x" ></i>
    </a>
</div>
"""
)

right_panel = ipw.VBox(
    [
        info_widget
        ,share_on_SNN_widget
        ,source_code_widget
        ,follow_author_widget
    ],
    layout = Layout(grid_area = 'right-panel')
)
right_panel.layout.align_items = 'center'
right_panel.layout.justify_content = 'space-around'
right_panel.layout.width = '100%'
# right_panel

In [15]:
app_header = ipw.HTML(
    value  ="""
    <div>
        <h1>Evolución de COVID-19 en España
        </h1>
    </div>
    """
    ,layout = Layout(grid_area = 'app-header')
)
app_header.add_class('app-header');

In [16]:
app = ipw.GridBox(children=[app_header, left_panel,right_panel],
        layout=Layout(
            grid_template_rows='auto auto auto',
            grid_template_columns='50% 50%',
            grid_template_areas='''
            "app-header app-header"
            "left-panel right-panel"
            ''')
       )
app

GridBox(children=(HTML(value='\n    <div>\n        <h1>Evolución de COVID-19 en España\n        </h1>\n    </d…